# Export Master File

Author: Cristian E. Nuno

Date: October 12, 2018

Purpose:

* unzips .zip files;
* reads them as separate data frames;
    + for both the `pitfail` and `prevent` worksheets
* binds them together into one data frame; and
* exports the results as a .csv file

In [1]:
# see the value of multiple statements at once
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# load necessary packages
import pandas as pd
import zipfile
import sys
import os
import re
import xlrd

# print version of python
print(sys.version)

3.5.6 |Anaconda custom (64-bit)| (default, Aug 26 2018, 16:30:03) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
# set working directory
directory = "/Users/cristiannuno/Desktop/Advanced_Analytics/DABP_Group_Project/"
raw_data = "raw_data/"
write_data = "write_data/"

os.chdir(directory + raw_data)

# load in states data frames
states = pd.read_csv("states.csv")

states.head()

# store abbreviations
state_abb = states["abb"]

,name,abb,region,division
0,Alabama,AL,South,East South Central
1,Alaska,AK,West,Pacific
2,Arizona,AZ,West,Mountain
3,Arkansas,AR,South,West South Central
4,California,CA,West,Pacific


In [3]:
# store all files in current wd
# sorting the items in the list alpabetically
file_names = sorted(os.listdir("."))

# create regex to only give back file names that contain .zip
my_pattern = re.compile(".zip$")

# store zip files that satisfy the regex
my_zip_files = list(filter(my_pattern.search, file_names))

In [4]:
# unzip each file in my_zip_files
for i in my_zip_files:
    print(i)
    zip = zipfile.ZipFile(file = i, mode = "r")
    zip.extractall(os.getcwd())
    zip.close()

2007_fsa_acres_sum_final.zip
2008_fsa_acres_sum_final.zip
2009_fsa_acres_sum_final_8.zip
2010_fsa_acres_sum_final_6.zip
2011_fsa_acres_sum_jan2012.zip
2012_fsa_acres_jan_2013.zip
2013_fsa_acres_jan_2014.zip
2014_fsa_acres_Jan2014.zip
2015_fsa_acres_Jan2016_sq19.zip
2016_fsa_acres_jan2017_edr32.zip
2017_fsa_acres_jan2018.zip


## Inspecting `.xlsx` files

After unzipping the files, I've manually opened them to inspect the `.xlsx` files. What I found was that each year contains two worksheets:

* `pltfail`: one row per state (including Puerto Rico and the Virgin Islands) representing the planted acres (including failed acres) reported to the Farm Service Agency for a variety of crops; and
* `prevent`: one row per state (including Puerto Rico and the Virgin Islands) representing the prevented acres reported to the Farm Service Agency for a variety of crops. 

This means we'll have two master files: one for planted and another for prevented acres from 2007 to 2017. *Note: not all of the workbooks spell these two sheets the same. However, each one starts with `pltfail`, followed by `prevent`.*

### Column Names

While the spelling of the column names are not all similar, thankfully, they are all in the correct order. Each year follows this pattern:
* `state`: the state name
* `barley`: acres for barley
* `corn`: acres for corn
* `cotton_els`: acres for cotton, extra long staple
* `cotton_upland`: acres for cotton, upland
* `oats`: acres for oats
* `rice`: acres for rice
* `sorghum`: acres for sorghum
* `sugar_beets`: acres for sugar beets
* `sugarcane`: acres for sugarcane
* `wheat`: acres for wheat
* `total`: sums the acres across all crops

There is no need for the `total` column so it'll be dropped.

### Note

The `.xlsx` files are not normalized in the sense that sheet names are spelled differently and the headers start at different rows. Going to manually import one data frame at a time.

In [5]:
# create regex to only give back file names that contain .xlsx
my_pattern = re.compile(".xlsx$")

# store xlsx files that satisfy the regex
my_xlsx_files = list(filter(my_pattern.search, file_names))

# store the first four digits from each item in the list
first_four_digits = []
for i in range(0, len(my_xlsx_files)):
    first_four_digits.append(int(my_xlsx_files[i][:4]))

# store column names
standard_col_names = ["state", "barley", "corn"
                     , "cotton_els", "cotton_upland", "oats"
                     , "rice", "sorghum", "soybeans"
                     , "sugar_beets", "sugarcane", "wheat"
                     , "total"]

# store non state names
non_states = ['Grand Total', 'US']

For each excel file, do the following:
* transfrom it into a data frame;
* standarize the column names;
* create a `year` column;
* create a `type` column to identify if this plant and fail data or prevention data;
* then row bind all the data frames together in `df`

In [6]:
# create an empty data frame
df = pd.DataFrame()
for i in range(0, len(my_xlsx_files)):
    # for years 2007 - 2011
    # make the header the second row in the file
    if i in list(range(0, 5)):
        header_row = 1
        
    # otherwise, make the header the fourth row in the file
    else:
        header_row = 3
    
    # print the current file being processed
    print(my_xlsx_files[i] + " is currently being processed...")
    
    # read in pltfail sheet
    pltfail = pd.read_excel(io = my_xlsx_files[i] \
                         , sheet_name = 0 \
                         , header = header_row)

    # standardize column names
    pltfail.columns = standard_col_names
    
    # create the year column
    pltfail["year"] = first_four_digits[i]
    
    # create the type column
    pltfail["type"] = "plant and fail"
    
    # read in prevent sheet
    prevent = pd.read_excel(io = my_xlsx_files[i] \
                         , sheet_name = 1 \
                         , header = header_row)
    
    # standardize column names
    prevent.columns = standard_col_names
    
    # create the year column
    prevent["year"] = first_four_digits[i]
    
    # create the type column
    prevent["type"] = "prevent"
    
    # append pltfail and prevent to temp
    temp = pltfail.append(prevent)
    
    # append temp to df
    df = df.append(temp)
    
    # print the current file that was binded to df
    print("... " + my_xlsx_files[i] + " was successfully binded to `df`.")
    
df.shape
df.columns
df["year"].value_counts()
df["type"].value_counts()

2007_fsa_acres_summary_final.xlsx is currently being processed...
... 2007_fsa_acres_summary_final.xlsx was successfully binded to `df`.
2008_fsa_acres_summary_final.xlsx is currently being processed...
... 2008_fsa_acres_summary_final.xlsx was successfully binded to `df`.
2009_fsa_acres_summary_final_8.xlsx is currently being processed...
... 2009_fsa_acres_summary_final_8.xlsx was successfully binded to `df`.
2010_fsa_acres_summary_final_6.xlsx is currently being processed...
... 2010_fsa_acres_summary_final_6.xlsx was successfully binded to `df`.
2011_fsa_acres_sum_jan2012.xlsx is currently being processed...
... 2011_fsa_acres_sum_jan2012.xlsx was successfully binded to `df`.
2012_fsa_acres_jan_2013.xlsx is currently being processed...
... 2012_fsa_acres_jan_2013.xlsx was successfully binded to `df`.
2013_fsa_acres_jan_2014.xlsx is currently being processed...
... 2013_fsa_acres_jan_2014.xlsx was successfully binded to `df`.
2014_fsa_acres_jan2014.xlsx is currently being processed.

(1174, 15)

Index(['state', 'barley', 'corn', 'cotton_els', 'cotton_upland', 'oats',
       'rice', 'sorghum', 'soybeans', 'sugar_beets', 'sugarcane', 'wheat',
       'total', 'year', 'type'],
      dtype='object')

2007    110
2015    108
2010    108
2008    108
2017    106
2016    106
2014    106
2013    106
2012    106
2009    106
2011    104
Name: year, dtype: int64

prevent           587
plant and fail    587
Name: type, dtype: int64

Now that we have our final data set, let's clean it up a bit:

* clean white space in the `state` column and make it title case;
* drop the `total` column;
* keep all rows where the `state` value doesn't equal `non_states`; and
* merge the `state` data frame onto `df_clean`.

In [7]:
# drop total
df_clean = df.drop("total", axis = 1)

# keep records of only states
df_clean = df_clean.query("state not in @non_states")
df_clean.shape

# keep certain rows
df_clean_abb = df_clean.query("state in @state_abb")
df_clean_nam = df_clean.query("state not in @state_abb")

# join states onto df_clean_abb
# remove the original state column
# and rename 'name' as state
df_clean_abb = pd.merge(df_clean_abb, states \
                        , how = "left", left_on = "state" \
                        , right_on = "abb")

df_clean_abb = df_clean_abb.drop("state", axis = 1)

df_clean_abb.columns = ["state" if x == "name" else x for x in df_clean_abb.columns]
df_clean_abb.shape

# clean the state column prior to merging
df_clean_nam["state"] = df_clean_nam["state"].str.strip().str.title()

# join states onto df_clean_nam
df_clean_nam = pd.merge(df_clean_nam, states \
                        , how = "left", left_on = "state" \
                        , right_on = "name")

# drop the name column
df_clean_nam = df_clean_nam.drop("name", axis = 1)
df_clean_nam.shape

# bind the two data frames together
df_clean = df_clean_abb.append(df_clean_nam)
df_clean.shape

(1152, 14)

(200, 17)

/Users/cristiannuno/ENTER/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(952, 17)

(1152, 17)

## Export data

Prior to exporting, rearrange the columns so that the data frame is more intuitive.

In [11]:
# set working directory
os.chdir(directory + write_data)

# reorder columns
# note: can only do this with integer positions rather than column names which is not ideal
new_order = [14, 16, 11, 0, 7, 5
            , 1, 2, 3, 4, 6, 8
            , 9, 10, 12, 13, 15]
#new_order = ["type", "year", "state", "abb", "region", "division"
#            , "barley", "corn", "cotton_els", "cotton_upland", "oats", "rice"
#            , "sorghum", "soybeans", "sugar_beets", "sugarcane", "wheat"]

df_clean = df_clean[df_clean.columns[new_order]]

df_clean.to_csv("clean_crop_2011_2017.csv", index = False)